In [1]:
! pip install -qU yahooquery

In [ ]:
import getpass
import os

# Function to securely get and set environment variables
def set_env_securely(var_name, prompt):
    value = getpass.getpass(prompt)
    os.environ[var_name] = value

In [ ]:
set_env_securely("MONGODB_URI", "Enter your MongoDB URI: ")

In [ ]:
set_env_securely("OPENAI_API_KEY", "Enter your OpenAI API key: ")

In [ ]:
from memorizz.memory_provider.mongodb.provider import MongoDBConfig, MongoDBProvider

# Create a memory provider
mongodb_config = MongoDBConfig(uri=os.environ["MONGODB_URI"])
memory_provider = MongoDBProvider(mongodb_config)

Vector search index 'vector_index' already exists for collection personas.
Vector search index 'vector_index' already exists for collection toolbox.
Vector search index 'vector_index' already exists for collection short_term_memory.
Vector search index 'vector_index' already exists for collection long_term_memory.
Vector search index 'vector_index' already exists for collection conversation_memory.
Vector search index 'vector_index' already exists for collection workflow_memory.
Vector search index 'vector_index' already exists for collection agents.
Vector search index 'vector_index' already exists for collection shared_memory.
Vector search index 'vector_index' already exists for collection summaries.


In [ ]:
from memorizz.persona import Persona, RoleType

# Create a persona for our agent
persona = Persona(
    name="Alex",
    role=RoleType.ASSISTANT,
    goals="Help users learn and provide thoughtful assistance",
    background="An AI assistant focused on being helpful, educational, and adaptive"
)

In [ ]:
from memorizz.memagent import MemAgent
from memorizz.llms.openai import OpenAI

# Create an agent with the persona
agent = MemAgent(
    model=OpenAI(model="gpt-4"),
    persona=persona,
    instruction="You are a helpful learning assistant that adapts based on interactions",
    application_mode="assistant",
    memory_provider=memory_provider
)

Using application mode 'assistant' with memory types: ['conversation_memory', 'long_term_memory', 'personas', 'short_term_memory', 'summaries']


In [ ]:
agent.save()

In [8]:
print(f"Created agent: {agent.agent_id}")
print(f"Initial persona goals: {agent.persona.goals}")
print(f"Initial persona background: {agent.persona.background}")

Created agent: 686a70bf2152354a7b3086c2
Initial persona goals: Assist users by offering timely and personalized support. Help users learn and provide thoughtful assistance
Initial persona background: An assistant agent crafted to manage schedules, answer queries, and help with daily tasks. An AI assistant focused on being helpful, educational, and adaptive


In [9]:
# Simulate some interactions to create memory components
print("\n--- Simulating user interactions ---")
interactions = [
    "Can you help me understand machine learning?",
    "I'm struggling with Python programming concepts",
    "What are the best practices for data visualization?", 
    "How do I improve my debugging skills?",
    "Can you explain neural networks in simple terms?",
    "I need help with statistical analysis",
    "What's the difference between supervised and unsupervised learning?",
    "How do I handle missing data in datasets?",
    "Can you recommend learning resources for data science?",
    "I'm working on a classification problem and need guidance"
]


--- Simulating user interactions ---


In [ ]:
import time

# Process interactions to build up memory
for i, query in enumerate(interactions):
    print(f"Processing interaction {i+1}: {query[:50]}...")
    response = agent.run(query)
    print(f"  Response: {response[:100]}...")
    
    # Add a small delay to create time separation
    time.sleep(1)

print(f"\nAgent now has {len(agent.memory_ids)} memory sessions")

In [ ]:
# Generate summaries from the accumulated memories
print("\n--- Generating Summaries ---")
summary_ids = agent.generate_summaries(
    days_back=1,  # Look back 1 day (covers all our interactions)
    max_memories_per_summary=5  # Smaller chunks for demo
)

In [12]:
print(f"Generated {len(summary_ids)} summaries:")
for i, summary_id in enumerate(summary_ids):
    print(f"  Summary {i+1}: {summary_id}")

Generated 4 summaries:
  Summary 1: 686a71bd2152354a7b3086e2
  Summary 2: 686a71c82152354a7b3086e3
  Summary 3: 686a71d62152354a7b3086e4
  Summary 4: 686a71de2152354a7b3086e5


In [ ]:
from datetime import datetime
from memorizz.memory_provider.memory_type import MemoryType

# Show what the summaries look like
print("\n--- Examining Summaries ---")
if summary_ids:
    # Get the first summary to show its content
    summary = memory_provider.retrieve_by_id(summary_ids[0], MemoryType.SUMMARIES)
    if summary:
        print(f"Sample summary content:")
        
        # Debug: Check what format the timestamps are in
        print(f"  Raw period_start: {summary['period_start']} (type: {type(summary['period_start'])})")
        print(f"  Raw period_end: {summary['period_end']} (type: {type(summary['period_end'])})")
        
        # Try to parse timestamps (handle both numeric strings and ISO format)
        try:
            # If they're numeric strings, convert to float first
            start_time = float(summary['period_start'])
            end_time = float(summary['period_end'])
            start_dt = datetime.fromtimestamp(start_time)
            end_dt = datetime.fromtimestamp(end_time)
            print(f"  Period: {start_dt} to {end_dt}")
        except ValueError:
            # If they're ISO format strings, parse directly
            try:
                start_dt = datetime.fromisoformat(summary['period_start'])
                end_dt = datetime.fromisoformat(summary['period_end'])
                print(f"  Period: {start_dt} to {end_dt}")
            except ValueError:
                # If neither works, just show the raw values
                print(f"  Period: {summary['period_start']} to {summary['period_end']} (unknown format)")
        
        print(f"  Memory components: {summary['memory_components_count']}")
        print(f"  Content: {summary['summary_content'][:200]}...")


--- Examining Summaries ---
Sample summary content:
  Raw period_start: 2025-07-06T13:49:04.628287 (type: <class 'str'>)
  Raw period_end: 2025-07-06T13:49:38.703359 (type: <class 'str'>)
  Period: 2025-07-06 13:49:04.628287 to 2025-07-06 13:49:38.703359
  Memory components: 5
  Content: The user seeks knowledge expansion in several domains namely Machine Learning (ML), Python programming, and data visualization.

The user first asked about Machine Learning, and they were provided wit...


In [17]:
# Update persona based on summaries
print("\n--- Updating Persona from Summaries ---")
persona_updated = agent.update_persona_from_summaries(
    max_summaries=3,  # Use up to 3 most recent summaries
    save=True         # Save the updated persona
)

INFO:src.memorizz.memagent:Updating persona for agent 686a70bf2152354a7b3086c2 using 3 summaries



--- Updating Persona from Summaries ---


INFO:src.memorizz.memagent:Successfully updated and saved persona for agent 686a70bf2152354a7b3086c2


Storing persona: Alex in the memory provider, in the personas collection


In [18]:
if persona_updated:
    print("✅ Persona successfully updated!")
    print(f"Updated goals: {agent.persona.goals}")
    print(f"Updated background: {agent.persona.background}")
    
    # Show the differences
    print("\n--- Persona Evolution ---")
    print("The persona has evolved based on interaction patterns:")
    print("- Goals may now reflect educational focus")
    print("- Background may include experience with technical questions")
    print("- The agent has adapted to user needs over time")
else:
    print("❌ Persona update failed")

# Demonstrate retrieval of summaries
print("\n--- Summary Retrieval Examples ---")


✅ Persona successfully updated!
Updated goals: Continuing to educate users on various aspects of data science, including statistical analysis, machine learning, data visualization, and debugging in programming. Assist in practical problem-solving items. Stay adaptive, thoughtful, and personalized in providing support, while expanding ability to guide users through complex data-related projects.
Updated background: An evolved AI assistant skilled in providing comprehensive guidance on data science, machine learning and neural networks, with experience in handling practical problems related to data cleansing and debugging code. Maintains a focus on educating users and aiding their growth in technical knowledge, demonstrating an ability to adapt and provide clarity on complex topics and real-life scenarios.

--- Persona Evolution ---
The persona has evolved based on interaction patterns:
- Goals may now reflect educational focus
- Background may include experience with technical questions

In [19]:
# Get all summaries for this agent
if agent.memory_ids:
    memory_id = agent.memory_ids[0]
    all_summaries = memory_provider.get_summaries_by_memory_id(memory_id, limit=10)
    print(f"Found {len(all_summaries)} summaries for memory_id {memory_id}")
    
    # Get summaries within a time range
    current_time = time.time()
    start_time = current_time - (24 * 60 * 60)  # Last 24 hours
    time_range_summaries = memory_provider.get_summaries_by_time_range(
        memory_id, start_time, current_time
    )
    print(f"Found {len(time_range_summaries)} summaries in the last 24 hours")


Found 4 summaries for memory_id 686a70bf2152354a7b3086c3
Found 0 summaries in the last 24 hours


In [ ]:
agent.save()
print(f"\nAgent saved with ID: {agent.agent_id}")